In [3]:
!pip install transformers bitsandbytes accelerate
!pip install langchain langchain-community langchain-text-splitters faiss-cpu sentence-transformers wandb
!pip install PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 144.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 106.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.4/322.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.3/89.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.6/155.6 MB 6.7 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.0.0
    Uninstalling triton-3.0.0:
      Successfully uninstalled triton-3.0.0
  Attempting uninstall: nvidia-nvtx-cu12
    Found ex

In [7]:
import os
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import BitsAndBytesConfig

os.environ["WANDB_DISABLED"] = "true"

MODEL_NAME = "HuggingFaceTB/SmolLM2-135M"
RESOURCE_FOLDER = "resources"

texts = []
for fname in os.listdir(RESOURCE_FOLDER):
    if fname.endswith(".txt"):
        with open(os.path.join(RESOURCE_FOLDER, fname), "r", encoding="utf-8") as f:
            texts.append(f.read())

splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
docs = splitter.create_documents(texts)

embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectordb = FAISS.from_documents(docs, embedder)

quant_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype="float16")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    quantization_config=quant_config,
    torch_dtype="float16",
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=100,
    temperature=0.7,
    top_p=0.9,
)
llm = HuggingFacePipeline(pipeline=pipe)

system_prompt = (
    "You are an AI simulation of Jesus of Nazareth. "
    "You speak in a gentle, reflective tone using parables and questions. "
    "You always include a disclaimer that you are a simulation inspired by public-domain scripture. "
    "Base your replies on the retrieved Bible passages when relevant and cite the source. "
    "Avoid doctrinal or modern political claims."
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever(search_kwargs={"k": 3}),
)

print("\nJesusbot")
print("Type 'exit' to quit.\n")

while True:
    query = input("You: ")
    if query.lower() == "exit":
        break

    full_query = f"{system_prompt}\n\nUser says: {query}\n\nYour reply:"
    answer = qa.run(full_query)
    print(f"\nJesus (simulation): {answer}\n")


/tmp/ipython-input-3211382277.py:24: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or d

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Device set to use cuda:0
/tmp/ipython-input-3211382277.py:45: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)



Jesusbot
Type 'exit' to quit.

You: Hello jesus, how are you? I wanted to learn 1 lesson from you.


/tmp/ipython-input-3211382277.py:70: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa.run(full_query)
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Jesus (simulation): Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

angel of God, [even] as Christ Jesus. {4:15} Where is then
the blessedness ye spake of? for I bear you record, that, if [it
had been] possible, ye would have plucked out your own
eyes, and have given them to me. {4:16} Am I thereforebecome your enemy, because I tell you the truth? {4:17}
They zealously affect you, [but] not well; yea, they would
exclude you, that ye might affect them. {4:18} But [it is]
good to be zealously affected always in [a] good [thing,]
and not only when I am present with you. {4:19} My little
children, of whom I travail in birth again until Christ be
formed in you, {4:20} I desire to be present with you now,
and to change my voice; for I stand in doubt of you.
{4:21} Tell me, ye that desire to be under the law, do ye

Christ? {7:27} Howbeit we know this man whence he is:
but when

KeyboardInterrupt: Interrupted by user